In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps 

# Import Google API key
from config import g_key 

# Configure gmaps
gmaps.configure(api_key=g_key) 

In [15]:
import pandas as pd 

df = pd.read_csv('WeatherPy_database.csv')

df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,61.20,67,40,13.80,scattered clouds
1,1,Lebu,CL,-37.6167,-73.6500,58.78,70,52,12.93,broken clouds
2,2,Sitka,US,57.0531,-135.3300,44.49,70,100,19.57,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,57.31,51,20,8.05,few clouds
4,4,Gubkinskiy,RU,64.4340,76.5026,-5.84,87,72,16.55,broken clouds


In [16]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [17]:
preferred_cities_df= df.loc[(df["Max Temp"] <= max_temp) & \
                            (df["Max Temp"] >= min_temp)]

In [18]:
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Hithadhoo,MV,-0.6000,73.0833,81.34,72,42,15.64,scattered clouds
14,14,Bonthe,SL,7.5264,-12.5050,79.09,80,8,5.44,clear sky
17,17,Atuona,PF,-9.8000,-139.0333,77.95,77,2,15.55,clear sky
18,18,Isangel,VU,-19.5500,169.2667,73.51,88,100,6.91,overcast clouds
23,23,Kapaa,US,22.0752,-159.3190,82.38,69,100,19.57,light rain
25,25,Rikitea,PF,-23.1203,-134.9692,74.01,86,65,18.72,broken clouds
27,27,Campana,AR,-34.1687,-58.9591,80.24,86,100,7.25,overcast clouds
29,29,Kulhudhuffushi,MV,6.6221,73.0700,81.16,73,73,6.60,broken clouds
30,30,Upington,ZA,-28.4478,21.2561,73.78,21,0,14.97,clear sky
37,37,Mount Isa,AU,-20.7333,139.5000,74.97,83,61,5.75,broken clouds


In [21]:
preferred_cities_df.count()

City_ID                11
City                   11
Country                11
Lat                    11
Lng                    11
Max Temp               11
Humidity               11
Cloudiness             11
Wind Speed             11
Current Description    11
dtype: int64

In [22]:
clean_travel_cities=preferred_cities_df.dropna()
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Hithadhoo,MV,-0.6000,73.0833,81.34,72,42,15.64,scattered clouds
14,14,Bonthe,SL,7.5264,-12.5050,79.09,80,8,5.44,clear sky
17,17,Atuona,PF,-9.8000,-139.0333,77.95,77,2,15.55,clear sky
18,18,Isangel,VU,-19.5500,169.2667,73.51,88,100,6.91,overcast clouds
23,23,Kapaa,US,22.0752,-159.3190,82.38,69,100,19.57,light rain
25,25,Rikitea,PF,-23.1203,-134.9692,74.01,86,65,18.72,broken clouds
27,27,Campana,AR,-34.1687,-58.9591,80.24,86,100,7.25,overcast clouds
29,29,Kulhudhuffushi,MV,6.6221,73.0700,81.16,73,73,6.60,broken clouds
30,30,Upington,ZA,-28.4478,21.2561,73.78,21,0,14.97,clear sky
37,37,Mount Isa,AU,-20.7333,139.5000,74.97,83,61,5.75,broken clouds


In [23]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()


hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Hithadhoo,MV,81.34,-0.6000,73.0833,
14,Bonthe,SL,79.09,7.5264,-12.5050,
17,Atuona,PF,77.95,-9.8000,-139.0333,
18,Isangel,VU,73.51,-19.5500,169.2667,
23,Kapaa,US,82.38,22.0752,-159.3190,
25,Rikitea,PF,74.01,-23.1203,-134.9692,
27,Campana,AR,80.24,-34.1687,-58.9591,
29,Kulhudhuffushi,MV,81.16,6.6221,73.0700,
30,Upington,ZA,73.78,-28.4478,21.2561,
37,Mount Isa,AU,74.97,-20.7333,139.5000,


In [24]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}      

In [25]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
#unable to get code to preform & unable to move forward with assignment 